# Pré-processamento para recursos numéricos

Neste notebook, ainda usaremos apenas recursos numéricos.

Apresentaremos esses novos aspectos:

* um exemplo de pré-processamento, nomeadamente **dimensionamento de variáveis numéricas**;
* usando um scikit-learn **pipeline** para encadear o pré-processamento e o modelo
  Treinamento;
* avaliando o desempenho estatístico do nosso modelo por meio de **cross-validation**
  em vez de uma única divisão de teste de trem.

## Preparação de dados

Primeiro, vamos carregar o conjunto de dados completo do censo adulto.

In [1]:
import pandas as pd
 
adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])
adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
# to display nice model diagram
from sklearn import set_config
set_config(display='diagram')

Vamos agora retirar o alvo dos dados que usaremos para treinar nosso
modelo preditivo.

In [3]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=target_name)

Em seguida, selecionamos apenas as colunas numéricas, como visto na anterior
caderno.

In [5]:
numerical_columns = [
    "age", "capital-gain:", "capital-loss:", "hours-per-week:"]

data_numeric = data[numerical_columns]

Finalmente, podemos dividir nosso conjunto de dados em conjuntos de treinamento e teste.

In [6]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data_numeric, target, random_state=42)

## Ajuste do modelo com pré-processamento

Uma variedade de algoritmos de pré-processamento no scikit-learn nos permite transformar
os dados de entrada antes de treinar um modelo. No nosso caso, vamos padronizar o
dados e, em seguida, treinar um novo modelo de regressão logística nessa nova versão do
o conjunto de dados.

Vamos começar imprimindo algumas estatísticas sobre os dados de treinamento.

In [7]:
data_train.describe()

,age,capital-gain:,capital-loss:,hours-per-week:
count,24420.000000,24420.000000,24420.000000,24420.000000
mean,38.598116,1045.622523,88.957207,40.386568
std,13.638458,7162.241991,405.633599,12.299621
min,17.000000,0.000000,0.000000,1.000000
25%,28.000000,0.000000,0.000000,40.000000
50%,37.000000,0.000000,0.000000,40.000000
75%,48.000000,0.000000,0.000000,45.000000
max,90.000000,99999.000000,4356.000000,99.000000


Vemos que os recursos do conjunto de dados abrangem diferentes intervalos. Algum
algoritmos fazem algumas suposições sobre as distribuições de recursos e
normalmente normalizar recursos será útil para lidar com essas suposições.

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p>Aqui estão alguns motivos para escalonar recursos:</p>
<ul class="last simple">
<li>Modelos que dependem da distância entre um par de amostras, por exemplo
vizinhos k-mais próximos, devem ser treinados em recursos normalizados para fazer cada
recurso contribui de forma aproximadamente igual para os cálculos de distância. </li>
<li> Muitos modelos, como regressão logística, usam um solucionador numérico (com base em
gradiente descendente) para encontrar seus parâmetros ideais. Este solucionador converge
mais rápido quando os recursos são dimensionados.</li>
</ul>
</div>

Se um modelo de aprendizado de máquina requer ou não o dimensionamento dos recursos depende
na família modelo. Modelos lineares, como regressão logística em geral
se beneficiam do dimensionamento dos recursos, enquanto outros modelos, como árvores de decisão
não precisa desse pré-processamento (mas não sofrerá com isso).

Mostramos como aplicar essa normalização usando um transformador scikit-learn
chamado `StandardScaler`. Este transformador muda e dimensiona cada recurso
individualmente para que todos tenham uma média 0 e um desvio padrão da unidade.

Investigaremos as diferentes etapas usadas no scikit-learn para alcançar tal
transformação dos dados.

Primeiro, é preciso chamar o método `fit`, a fim de aprender o escalonamento de
os dados.

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data_train)

StandardScaler()

O método `fit` para transformadores é semelhante ao método` fit` para
preditores. A principal diferença é que o primeiro tem um único argumento (o
matriz de dados), enquanto o último tem dois argumentos (a matriz de dados e o
alvo).

![Predictor fit diagram](imagens/transformer.fit_.png)

Neste caso, o algoritmo precisa calcular a média e o desvio padrão
para cada recurso e armazene-os em alguns arrays NumPy. Aqui, estes
as estatísticas são os estados do modelo.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">O fato de que os estados do modelo deste scaler são matrizes de meios e
desvios padrão são específicos para o <tt class="docutils literal">StandardScaler</tt>. Outro
Os transformadores scikit-learn irão calcular diferentes estatísticas e armazená-las
como o modelo afirma, da mesma maneira.</p>
</div>

Podemos inspecionar as médias calculadas e os desvios-padrão.

In [9]:
scaler.mean_

array([  38.5981163 , 1045.62252252,   88.95720721,   40.38656839])

In [10]:
scaler.scale_

array([  13.63817856, 7162.09534233,  405.62529379,   12.29936908])

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">scikit-learn convenção: se um atributo é aprendido a partir dos dados, seu nome
termina com um sublinhado (i.e. <tt class="docutils literal">_</tt>), as in <tt class="docutils literal">mean_</tt> and <tt class="docutils literal">scale_</tt> for the
<tt class="docutils literal">StandardScaler</tt>.</p>
</div>

O dimensionamento dos dados é aplicado a cada recurso individualmente (ou seja, cada coluna em
a matriz de dados). Para cada recurso, subtraímos sua média e dividimos por seu
desvio padrão.

Depois de chamar o método `fit`, podemos realizar a transformação de dados por
chamando o método `transform`.

In [11]:
data_train_scaled = scaler.transform(data_train)
data_train_scaled

array([[ 0.76270329, -0.14599394, -0.21930883, -0.03142994],
       [-0.85041534, -0.14599394, -0.21930883,  0.37509498],
       [ 0.39608542, -0.14599394, -0.21930883,  1.59466973],
       ...,
       [-1.51032751, -0.14599394, -0.21930883, -1.65752961],
       [ 0.83602687, -0.14599394, -0.21930883,  3.54598934],
       [-0.33715032, -0.14599394, -0.21930883,  1.59466973]])

Vamos ilustrar o mecanismo interno do método `transform` e colocá-lo
para perspectiva com o que já vimos com preditores.

![Predictor fit diagram](imagens/transformer.transform_.png)

O método `transform` para transformadores é semelhante ao método` predict`
para preditores. Ele usa uma função predefinida, chamada de **transformation
function** e usa os estados do modelo e os dados de entrada. No entanto, em vez de
produzindo previsões, o trabalho do método `transform` é produzir um
versão transformada dos dados de entrada.

Finalmente, o método `fit_transform` é um método abreviado para chamar
sucessivamente `fit` e, em seguida,` transform`.

![Predictor fit diagram](imagens/transformer.fit_transform.png)



In [12]:
data_train_scaled = scaler.fit_transform(data_train)
data_train_scaled

array([[ 0.76270329, -0.14599394, -0.21930883, -0.03142994],
       [-0.85041534, -0.14599394, -0.21930883,  0.37509498],
       [ 0.39608542, -0.14599394, -0.21930883,  1.59466973],
       ...,
       [-1.51032751, -0.14599394, -0.21930883, -1.65752961],
       [ 0.83602687, -0.14599394, -0.21930883,  3.54598934],
       [-0.33715032, -0.14599394, -0.21930883,  1.59466973]])

In [13]:
data_train_scaled = pd.DataFrame(data_train_scaled,
                                 columns=data_train.columns)
data_train_scaled.describe()

,age,capital-gain:,capital-loss:,hours-per-week:
count,2.442000e+04,2.442000e+04,2.442000e+04,2.442000e+04
mean,2.070029e-16,8.018906e-16,1.849333e-15,1.202251e-15
std,1.000020e+00,1.000020e+00,1.000020e+00,1.000020e+00
min,-1.583651e+00,-1.459939e-01,-2.193088e-01,-3.202324e+00
25%,-7.770918e-01,-1.459939e-01,-2.193088e-01,-3.142994e-02
50%,-1.171796e-01,-1.459939e-01,-2.193088e-01,-3.142994e-02
75%,6.893797e-01,-1.459939e-01,-2.193088e-01,3.750950e-01
max,3.768970e+00,1.381626e+01,1.051967e+01,4.765564e+00


Podemos facilmente combinar essas operações sequenciais com um scikit-learn
`Pipeline`, que encadeia operações e é usado como qualquer outro
classificador ou regressor. A função auxiliar `make_pipeline` irá criar um
`Pipeline`: toma como argumentos as transformações sucessivas a serem realizadas,
seguido pelo classificador ou modelo regressor.

In [14]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), LogisticRegression())
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

A função `make_pipeline` não nos obrigou a dar um nome para cada etapa.
Na verdade, foi atribuído automaticamente com base no nome das classes
forneceu; um `StandardScaler` será um passo denominado` "standardscaler" `no
pipeline resultante. Podemos verificar o nome de cada etapa do nosso modelo:

In [15]:
model.named_steps

{'standardscaler': StandardScaler(),
 'logisticregression': LogisticRegression()}

Este pipeline preditivo expõe os mesmos métodos do preditor final:
`fit` e` Predict` (e adicionalmente `Predict_proba`,` Decision_function`,
ou `score`).

In [16]:
start = time.time()
model.fit(data_train, target_train)
elapsed_time = time.time() - start

Podemos representar o mecanismo interno de um pipeline ao chamar `fit`
pelo seguinte diagrama:

![Predictor fit diagram](imagens/pipeline.fit.png)

Ao chamar `model.fit`, o método` fit_transform` de cada subjacente
transformador (aqui um único transformador) no pipeline será chamado para:

- aprender seus estados de modelo interno
- transformar os dados de treinamento. Finalmente, os dados pré-processados são fornecidos para
  treinar o preditor.

Para prever os alvos dado um conjunto de teste, usa-se o método `predict`.

In [17]:
predicted_target = model.predict(data_test)
predicted_target[:5]

array(['<=50K', '<=50K', '<=50K', '<=50K', '<=50K'], dtype=object)

Vamos mostrar o mecanismo subjacente:

![Predictor fit diagram](imagens/pipeline.predict_.png)

O método `transform` de cada transformador (aqui um único transformador) é
chamado para pré-processar os dados. Observe que não há necessidade de chamar o `fit`
método para esses transformadores porque estamos usando os estados do modelo interno
calculado ao chamar `model.fit`. Os dados pré-processados são então fornecidos para
o preditor que produzirá o alvo previsto chamando seu método
`predict`.

Em resumo, podemos verificar a pontuação do pipeline preditivo completo
chamando o método `model.score`. Assim, vamos verificar o computacional e
desempenho estatístico de tal pipeline preditivo.

In [20]:
model_name = model.__class__.__name__
score = model.score(data_test, target_test)
print(f"A precisão usando um {model_name} é {score:.3f} "
      f"com um tempo adequado de {elapsed_time:.3f} seconds "
      f"no {model[-1].n_iter_[0]} iterações")

A precisão usando um Pipeline é 0.801 com um tempo adequado de 0.082 seconds no 12 iterações


Podemos comparar este modelo preditivo com o modelo preditivo usado em
o notebook anterior que não escalou recursos.

In [21]:
model = LogisticRegression()
start = time.time()
model.fit(data_train, target_train)
elapsed_time = time.time() - start

In [22]:
model_name = model.__class__.__name__
score = model.score(data_test, target_test)
print(f"A precisão usando um {model_name} é {score:.3f} "
      f"com um tempo adequado de {elapsed_time:.3f} seconds "
      f"no {model.n_iter_[0]} iterações")

A precisão usando um LogisticRegression é 0.801 com um tempo adequado de 0.270 seconds no 62 iterações


Vemos que dimensionar os dados antes de treinar a regressão logística foi
benéfico em termos de desempenho computacional. Na verdade, o número de
as iterações diminuíram, assim como o tempo de treinamento. O estatístico
o desempenho não mudou, uma vez que ambos os modelos convergiram.

<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Warning</p>
<p class="last">Trabalhar com dados não escalonados forçará potencialmente o algoritmo a iterar
mais como mostramos no exemplo acima. Há também o catastrófico
cenário em que o número de iterações necessárias é maior que o máximo
número de iterações permitidas pelo preditor (controlado pelo<tt class="docutils literal">max_iter</tt>)parâmetro. Portanto, antes de aumentar <tt class = "docutils literal"> max_iter </tt>, certifique-se de que os dados
são bem dimensionados.</p>
</div>

## Avaliação do modelo usando validação cruzada

No exemplo anterior, dividimos os dados originais em um conjunto de treinamento e um
conjunto de teste. Essa estratégia tem vários problemas: em um cenário onde a quantidade
de dados é pequeno, o subconjunto usado para treinar ou testar será pequeno. Além disso, um
a divisão única não fornece informações sobre a confiança do
resultados obtidos.

Em vez disso, podemos usar validação cruzada. A validação cruzada consiste em repetir
o procedimento de modo que os conjuntos de treinamento e teste sejam diferentes cada
Tempo. As métricas de desempenho estatísticas são coletadas para cada repetição e
em seguida, agregados. Como resultado, podemos obter uma estimativa da variabilidade do
desempenho estatístico do modelo.

Observe que existem várias estratégias de validação cruzada, cada uma delas
define como repetir o procedimento `fit` /` score`. Nesta seção, vamos
use a estratégia K-fold: todo o conjunto de dados é dividido em partições `K`. O
O procedimento `fit` /` score` é repetido `K` vezes onde em cada iteração` K - 1`
as partições são usadas para ajustar o modelo e a partição `1` é usada para pontuar. O
a figura abaixo ilustra essa estratégia K-fold.

![Predictor fit diagram](imagens/k_fold.png)

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Esta figura mostra o caso particular da estratégia de validação cruzada K-fold.
Conforme mencionado anteriormente, há uma variedade de diferentes tipos de validação cruzada
estratégias. Alguns desses aspectos serão abordados com mais detalhes no futuro
cadernos.</p>
</div>

Para cada divisão de validação cruzada, o procedimento treina um modelo em todos os
amostras e avaliar a pontuação do modelo nas amostras azuis.
A validação cruzada é, portanto, computacionalmente intensiva porque requer
treinar vários modelos em vez de um.

No scikit-learn, a função `cross_validate` permite fazer validação cruzada
e você precisa passar o modelo, os dados e o destino. Desde então
existe várias estratégias de validação cruzada, `cross_validate` leva um
parâmetro `cv` que define a estratégia de divisão.

In [23]:
%%time
from sklearn.model_selection import cross_validate

model = make_pipeline(StandardScaler(), LogisticRegression())
cv_result = cross_validate(model, data_numeric, target, cv=5)
cv_result

Wall time: 875 ms


{'fit_time': array([0.11964297, 0.13605833, 0.1206336 , 0.13871574, 0.140939  ]),
 'score_time': array([0.03006649, 0.0272398 , 0.03157139, 0.02211022, 0.02812767]),
 'test_score': array([0.80454476, 0.79560811, 0.80128993, 0.7972973 , 0.80420762])}

A saída de `cross_validate` é um dicionário Python, que por padrão
contém três entradas: (i) o tempo para treinar o modelo nos dados de treinamento
para cada dobra, (ii) o tempo para prever com o modelo nos dados de teste
para cada dobra e (iii) a pontuação padrão nos dados de teste para cada dobra.

Definir `cv = 5` criou 5 divisões distintas para obter 5 variações para o treinamento
e conjuntos de teste. Cada conjunto de treinamento é usado para ajustar um modelo que é então
pontuado no conjunto de teste correspondente. Esta estratégia é chamada K-fold
validação cruzada onde `K` corresponde ao número de divisões.

Observe que, por padrão, a função `cross_validate` descarta os 5 modelos que
foram treinados em diferentes subconjuntos sobrepostos do conjunto de dados. O objetivo de
validação cruzada não é treinar um modelo, mas sim estimar
aproximadamente o desempenho de generalização de um modelo que teria sido
treinados para o conjunto de treinamento completo, juntamente com uma estimativa da variabilidade
(incerteza sobre a precisão da generalização).

Você pode passar parâmetros adicionais para `cross_validate` para obter mais
informações, por exemplo, pontuações de treinamento. Esses recursos serão abordados em
um futuro caderno.

Vamos extrair as pontuações do teste do dicionário `cv_result` e calcular
a precisão média e a variação da precisão nas dobras.

In [24]:
scores = cv_result["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.801 +/- 0.004


Observe que, ao calcular o desvio padrão das pontuações de validação cruzada,
podemos estimar a incerteza do desempenho estatístico do nosso modelo. Isso é
a principal vantagem da validação cruzada e pode ser crucial na prática, para
exemplo ao comparar diferentes modelos para descobrir se um é melhor
do que o outro ou se as diferenças de desempenho estatístico estão dentro
A incerteza.

Neste caso particular, apenas as 2 primeiras casas decimais parecem ser confiáveis. Se
você sobe neste notebook, você pode verificar se o desempenho que obtemos
com validação cruzada é compatível com o de um único teste de trem
dividir.

Neste caderno temos:

* viu a importância de **scaling numerical variables**;
* usou um **pipeline** para escalonamento da cadeia e treinamento de regressão logística;
* avaliou o desempenho estatístico do nosso modelo por meio de **cross-validation**.